In [ ]:
# Uncomment these lines if needed:
!pip install nltk requests transformers
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

import requests
import re
import time
import json
from urllib.parse import quote

#########################
# 1. Transformer Baselines
#########################
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer

def baseline_generate_t5(question, num_candidates=5):
    """
    Generates candidate answers for a question using T5 (without any external knowledge).
    """
    model_name = "t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    input_text = "question: " + question
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=num_candidates,
        do_sample=True,
        top_k=50
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

def baseline_generate_bart(question, num_candidates=5):
    """
    Generates candidate answers for a question using BART (without knowledge integration).
    """
    model_name = "facebook/bart-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    input_ids = tokenizer.encode(question, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=num_candidates,
        num_beams=num_candidates,
        do_sample=True,
        top_k=50
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

def baseline_generate_gpt2(question, num_candidates=5):
    """
    Generates candidate answers for a question using GPT-2 (without knowledge integration).
    """
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    encoded = tokenizer(question, return_tensors="pt")
    input_ids = encoded["input_ids"]
    attention_mask = encoded["attention_mask"]
    outputs = model.generate(
         input_ids,
         attention_mask=attention_mask,
         max_length=50,
         num_return_sequences=num_candidates,
         num_beams=num_candidates,
         do_sample=False,
         pad_token_id=tokenizer.eos_token_id
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


#########################
# 2. NLTK Utilities for Lemmatization, POS Tagging, and Expansion
#########################
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return set(lemmas)

def expand_word(word):
    expansion = set([word])
    for syn in wn.synsets(word):
        for lemma in syn.lemma_names():
            expansion.add(lemma.lower().replace("_", " "))
        for hyper in syn.hypernyms():
            for lemma in hyper.lemma_names():
                expansion.add(lemma.lower().replace("_", " "))
    return expansion

def expand_tokens(token_set):
    expanded = set()
    for token in token_set:
        expanded.update(expand_word(token))
    return expanded

def determine_expected_answer_type(question):
    """
    Simple heuristic: if the question contains "do", "perform", or "act", expect an action; else an object.
    """
    tokens = nltk.word_tokenize(question.lower())
    if any(word in tokens for word in ["do", "perform", "act"]):
        return "action"
    return "object"

#########################
# 3. Improved Candidate Generation Using ConceptNet
#########################
BASE_URL = "http://api.conceptnet.io"

def get_edges_for_term(term, limit=50):
    url = f"{BASE_URL}{term}?offset=0&limit={limit}"
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        data = resp.json()
        return data.get("edges", [])
    except requests.RequestException as e:
        print(f"Error fetching ConceptNet data for {term}: {e}")
        return []

def text_to_conceptnet_uri(text, lang='en'):
    text_underscored = text.strip().lower().replace(" ", "_")
    return f"/c/{lang}/{quote(text_underscored)}"

def extract_keywords_improved(question_text, expected_type):
    """
    Improved keyword extraction:
      - For actions, extract verbs but filter out trivial ones.
      - For objects, extract nouns.
      - If too few, fallback to all content words.
    """
    tokens = nltk.word_tokenize(question_text.lower())
    pos_tags = nltk.pos_tag(tokens)
    stopwords = {"what", "name", "something", "you", "on", "the", "a", "an", "of"}

    if expected_type == "action":
        trivial_verbs = {"name", "do", "get", "be", "have"}
        keywords = [word for word, tag in pos_tags if tag.startswith("VB") and word not in trivial_verbs]
        if len(keywords) < 1:
            keywords = [word for word in tokens if word not in stopwords]
    else:
        keywords = [word for word, tag in pos_tags if tag.startswith("NN")]
        if len(keywords) < 1:
            keywords = [word for word in tokens if word not in stopwords]

    return keywords

def filter_candidates_by_pos(candidates, expected_type):
    """
    For "action", require at least one verb; for "object", require at least one noun.
    """
    filtered = []
    for candidate in candidates:
        tokens = nltk.word_tokenize(candidate)
        pos_tags = nltk.pos_tag(tokens)
        if expected_type == "action":
            if any(tag.startswith("VB") for _, tag in pos_tags):
                filtered.append(candidate)
        else:
            if any(tag.startswith("NN") for _, tag in pos_tags):
                filtered.append(candidate)
    return filtered

def generate_candidates_improved(question_text, top_k=20):
    """
    Candidate generation using ConceptNet, with broader relation filters and contextual cues.
    """
    expected_type = determine_expected_answer_type(question_text)
    keywords = extract_keywords_improved(question_text, expected_type)

    # If too few keywords, fallback to all content words
    if len(keywords) < 1:
        tokens = nltk.word_tokenize(question_text.lower())
        stopwords = {"what", "name", "something", "you", "on", "the", "a", "an", "of", "do"}
        keywords = [word for word in tokens if word not in stopwords]

    # Add contextual cues: if the question mentions "rain", add related words.
    if "rain" in question_text.lower():
        contextual_cues = ["rain", "rainy", "weather", "wet", "drizzle"]
        keywords = list(set(keywords).union(set(contextual_cues)))

    # Relaxed relation filters:
    if expected_type == "action":
        relation_set = {
            "/r/UsedFor", "/r/CapableOf", "/r/MotivatedBy", "/r/Desires",
            "/r/RelatedTo", "/r/HasSubevent", "/r/HasPrerequisite"
        }
    else:
        relation_set = {
            "/r/IsA", "/r/PartOf", "/r/HasA", "/r/RelatedTo", "/r/AtLocation"
        }

    candidates = set()
    for kw in keywords:
        uri = text_to_conceptnet_uri(kw)
        edges = get_edges_for_term(uri, limit=top_k)
        for e in edges:
            rel_id = e.get("rel", {}).get("@id", "")
            if rel_id in relation_set:
                end_id = e.get("end", {}).get("@id", "")
                start_id = e.get("start", {}).get("@id", "")
                if start_id == uri and end_id.startswith("/c/en/"):
                    candidates.add(end_id)
                elif end_id == uri and start_id.startswith("/c/en/"):
                    candidates.add(start_id)

    text_candidates = set()
    for c in candidates:
        cleaned = c.replace("/c/en/", "").replace("_", " ")
        text_candidates.add(cleaned)

    # Filter candidates by expected POS
    filtered_candidates = filter_candidates_by_pos(list(text_candidates), expected_type)
    return filtered_candidates


#########################
# 4. Answer Ranking
#########################
def score_brevity(answer):
    tokens = answer.split()
    return 1.0 / len(tokens) if tokens else 0.0

def get_concreteness(answer):
    # Example dictionary for demonstration
    concreteness_dict = {
        'apple': 5.0,
        'cherry': 4.5,
        'blueberry': 4.0,
        'watermelon': 4.0,
        'lemon': 4.0,
        'lime': 4.0,
        'pineapple': 4.0,
        'peach': 4.0,
        'raspberry': 4.0,
        'orange': 4.0,
        'grape': 4.0,
        'mango': 4.0,
        'plums': 4.0,
    }
    tokens = answer.lower().split()
    scores = [concreteness_dict.get(token, 3.0) for token in tokens]
    return sum(scores) / len(scores) if scores else 0.0

def score_typicality(question, answer):
    q_tokens = tokenize_and_lemmatize(question)
    a_tokens = tokenize_and_lemmatize(answer)
    q_expanded = expand_tokens(q_tokens)
    a_expanded = expand_tokens(a_tokens)
    if not q_expanded or not a_expanded:
        return 0.0
    intersection = q_expanded.intersection(a_expanded)
    union = q_expanded.union(a_expanded)
    return len(intersection) / len(union)

def rank_candidates_by_composite(question, candidates, weights=(0.3, 0.4, 0.3)):
    """
    Ranks candidate answers by a weighted sum of brevity, concreteness, and typicality.
    """
    ranked = []
    for candidate in candidates:
        brevity = score_brevity(candidate)
        concreteness = get_concreteness(candidate)
        typicality = score_typicality(question, candidate)
        composite_score = weights[0]*brevity + weights[1]*concreteness + weights[2]*typicality
        ranked.append((candidate, composite_score, brevity, concreteness, typicality))
    ranked.sort(key=lambda x: x[1], reverse=True)
    return ranked


#########################
# 5. Enhanced Post-Processing
#########################
import difflib

def postprocess_candidates(ranked_candidates, question, similarity_threshold=0.8):
    """
    - Removes near-duplicates (redundancy removal).
    - Performs a basic coherence check:
      * Removes extremely short or nonsensical answers.
    """
    final = []
    existing = []  # Accepted answers for duplication check

    for cand_tuple in ranked_candidates:
        candidate, comp_score, brevity, conc, typ = cand_tuple

        # Basic length check: avoid extremely short answers.
        if len(candidate.strip()) < 2:
            continue

        # Near-duplicate check using difflib.
        is_duplicate = False
        for accepted in existing:
            ratio = difflib.SequenceMatcher(None, candidate.lower(), accepted.lower()).ratio()
            if ratio >= similarity_threshold:
                is_duplicate = True
                break
        if is_duplicate:
            continue

        final.append(cand_tuple)
        existing.append(candidate)

    return final


#########################
# 6. Main Pipeline
#########################
def main():
    sample_data = [
        {
            "question": "Name a fruit you might put in a pie",
            "answers": ["apple", "cherry", "blueberry"]
        },
        {
            "question": "Name something you might do at a party",
            "answers": ["dance", "talk", "mingle"]
        },
        {
            "question": "Name something you might find in a kitchen",
            "answers": ["knife", "fork", "spoon"]
        }
    ]

    for entry in sample_data:
        question = entry["question"]
        gold_answers = entry.get("answers", [])
        print("\n==============================")
        print(f"Processing Question: {question}")
        print("==============================")

        # 1) Baseline Transformer Outputs
        print("\nBaseline Evaluation (without knowledge integration):")
        print("\nT5 Generated Answers:")
        for ans in baseline_generate_t5(question):
            print(" -", ans)
        print("\nBART Generated Answers:")
        for ans in baseline_generate_bart(question):
            print(" -", ans)
        print("\nGPT-2 Generated Answers:")
        for ans in baseline_generate_gpt2(question):
            print(" -", ans)

        # 2) Candidate Generation using ConceptNet
        print("\nCandidate Generation Using ConceptNet:")
        candidates = generate_candidates_improved(question, top_k=20)
        if not candidates:
            print("No candidates found.")
            print("\nGold Answers (for reference):", gold_answers)
            continue
        print("Generated Candidates (raw):")
        print(candidates)

        # 3) Ranking
        ranked_candidates = rank_candidates_by_composite(question, candidates)
        print("\nRanked Candidates (candidate | composite | brevity | concreteness | typicality):")
        for cand, comp, brev, conc, typ in ranked_candidates:
            print(f"{cand:30s} | {comp:8.3f} | {brev:7.3f} | {conc:11.3f} | {typ:10.3f}")

        # 4) Enhanced Post-Processing
        print("\nPost-Processing for Redundancy & Coherence:")
        final_candidates = postprocess_candidates(ranked_candidates, question, similarity_threshold=0.8)
        if not final_candidates:
            print("All candidates removed during post-processing.")
        else:
            print("Final Post-Processed Candidates:")
            for cand, comp, brev, conc, typ in final_candidates:
                print(f"{cand:30s} | {comp:8.3f} | {brev:7.3f} | {conc:11.3f} | {typ:10.3f}")

        print("\nGold Answers (for reference):")
        print(gold_answers)

        time.sleep(0.5)

        time.sleep(0.5)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.



Processing Question: Name a fruit you might put in a pie

Baseline Evaluation (without knowledge integration):

T5 Generated Answers:
 - Identify an fruit you might put in a pie
 - don't forget your name.
 - name a fruit as a fruit you might put in a pie
 - names
 - nom

BART Generated Answers:
 - Name a fruit you might put in a pie
 - Name a fruit you might put in a pie.
 - Name a fruit you might put in a pieadvertisement
 - Name a fruit you might put in a pie;
 - Name a fruit you might put in a pie (

GPT-2 Generated Answers:
 - Name a fruit you might put in a pie crust.

You can also make your own pie crust by using a pastry brush.

You can also make your own pie crust by using a pastry brush. You can also make your own pie
 - Name a fruit you might put in a pie crust.

You can also make your own pie crust by using a pastry brush.

You can also make your own pie crust by using a pastry brush.

You can also make your
 - Name a fruit you might put in a pie crust.

You can also make y